In [1]:
%defaultDatasource jdbc:h2:mem:db

# Modelagem

## Modelo Conceitual
![Modelo Conceitual](../img/conceitual-v3.png "Title")

## Modelo Lógico

- **Disease**(*name*, occurrences)
- **Symptom**(*name*, occurrences)
- **Cause**(*disease*, *symptom*, occurrences, score)
    - Chave Estrangeira: *disease* -> **Disease**
    - Chave Estrangeira: *symptom* -> **Symptom**
- **Similarity**(disease_from, disease_to, score)
    - Chave Estrangeira: *disease_from* -> **Disease**
    - Chave Estrangeira: *disease_to* -> **Disease**
    
- **HighSimilarity**(disease_from, disease_to, score)
    - Chave Estrangeira: *disease_from* -> **Disease**
    - Chave Estrangeira: *disease_to* -> **Disease**


# Criação das Tabelas Relacionais

```
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/esdrasbrz/mc536-sugoi/master/data/ncomms-disease.csv" AS line
FIELDTERMINATOR '|'
CREATE (d:Disease {name: line.name, occurrences: toInt(line.occurrences)})

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/esdrasbrz/mc536-sugoi/master/data/ncomms-symptom.csv" AS line
FIELDTERMINATOR '|'
CREATE (s:Symptom {name: line.name, occurrences: toInt(line.occurrences)})

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/esdrasbrz/mc536-sugoi/master/data/ncomms-cause.csv" AS line
FIELDTERMINATOR '|'
MATCH (s:Symptom {name: line.symptom})
MATCH (d:Disease {name: line.disease})
CREATE (d)-[:CAUSES {occurrences: line.occurrences, score: toFloat(line.score)}]->(s)

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/esdrasbrz/mc536-sugoi/master/data/ncomms-similarity.csv" AS line
FIELDTERMINATOR '|'
MATCH (f:Disease {name: line.disease_from})
MATCH (t:Disease {name: line.disease_to})
CREATE (f)-[:SIMILAR_TO {score: toFloat(line.score)}]->(t)

MATCH (f:Disease)-[sim:SIMILAR_TO]->(t:Disease)
WHERE sim.score >= 0.8
CREATE (f)-[:VERY_SIMILAR_TO {score: sim.score}]->(t)
```

# Análise de Dados

### Doenças que possuem o sintoma "Amnesia"

Listagem das doenças que possuem o sintoma amnésia e suas relações de similaridade. Considerar apenas doenças com score acima de 200 para esse sintoma.
    
```
MATCH (d:Disease)-[c:CAUSES]->(s:Symptom {name: "Amnesia"})
WHERE c.score > 200
RETURN d, s
```

![Sintoma Amnesia](../img/sintoma_amnesia.png "Title")

### Similaridade de doenças com Diabetes Tipo 1 em dois níveis

Listagem das doenças muito similares (VERY_SIMILAR_TO) com Diabetes Tipo 1 em dois níveis de hierarquia, isso é, as doenças similares diretamente a Diabetes e as doenças similares às similares à Diabetes. A query pode ser extendida para mais de dois níveis.
    
```
MATCH (d:Disease {name: "Diabetes Mellitus, Type 1")-[:VERY_SIMILAR_TO*1..2]-(t:Disease)
RETURN d, t
```

![Similaridade 2 níveis](../img/similaridade_2_niveis.png "Title")

### Listar todas as doenças da componente conexa da Diabetes Tipo 1

Primeiro faz a listagem de componentes conexas e retorna o id da componente em que se encontra a Diabetes Tipo 1.
    
```
CALL algo.unionFind.stream('Disease', 'VERY_SIMILAR_TO', {})
YIELD nodeId, setId

MATCH (d:Disease) 
WHERE id(d) = nodeId AND d.name = "Diabetes Mellitus, Type 1"
RETURN d.name AS disease, setId
```

Salvo o setId, podemos listar as doenças desse conjunto:

```
CALL algo.unionFind.stream('Disease', 'VERY_SIMILAR_TO', {})
YIELD nodeId, setId

MATCH (d:Disease)-[r:VERY_SIMILAR_TO]-(:Disease) 
WHERE id(d) = nodeId AND setId = 12
RETURN d, r
```

![Componente Conexa](../img/diabetes_componente_conexa.png "Title")

### Usar algoritmo Label Propagation para encontrar clusters de doenças altamente similares

Primeiramente roda-se o algoritmo setando a propriedade partition como o id do cluster encontrado em cada doença

```
CALL algo.labelPropagation.stream("Disease", "VERY_SIMILAR_TO", {iterations:1, weightProperty:'score', partitionProperty:'partition', concurrency:4, direction:'BOTH'})
YIELD nodeId, label

MATCH (d:Disease)
WHERE id(d) = nodeId
SET d.partition = label
```

Agora, a partir de uma doença encontra-se as outras pertencentes ao mesmo cluster que ela. No exemplo, usaremos "Kidney Diseases".

```
MATCH (a:Disease {name: "Kidney Diseases"})

MATCH (d:Disease)
WHERE d.partition = a.partition
RETURN d
```

![Kidney Cluster](../img/kidney_cluster.png "Title")

### Algoritmo PageRank para identificar as doenças com mais conexões entre similares

Roda-se o algoritmo PageRank nas doenças e em suas relações VERY_SIMILAR_TO e faz a listagem das doenças com maior score.

```
CALL algo.pageRank.stream("Disease", "VERY_SIMILAR_TO",
    {iterations:20, dampingFactor:0.85, concurrency:4})
YIELD nodeId, score

MATCH (d:Disease)
WHERE id(d) = nodeId

RETURN d.name AS disease, score
ORDER BY score DESC
```

| **Disease** | **Score** |
|-------------|-----------|
|Blepharophimosis|3.4534739999999995|
|Orofaciodigital Syndromes|3.1682224999999997|
|Hypoalphalipoproteinemias|2.3670379999999995|
|Fibromatosis, Gingival|1.9850735000000002|
|Chromosome Breakage|1.6526724999999998|